In [ ]:
from find_source import start_html, catalog_to_html, fig_to_html, end_html, make_catalog, combine_catalogs
import glob
import pandas as pd

In [ ]:
start_html()
final_catalog = {}
for file in glob.glob('../data/new_250609_17:30:22/*.fits'):
    try:
        final_catalog = combine_catalogs(final_catalog, make_catalog(file))
    except:
        print(f'Try again for {file}')
catalog_to_html(final_catalog)
for file in glob.glob('../data/250611_03:56:34/*.fits'):
    try:
        fig_to_html(file)
    except:
        print(f'Try again for {file}')
end_html()